Evaluación de desempeño

utilizaremos un Processing Job de Amazon SageMaker pero en este caso para buscar el umbral de clasificación

Esto lo haremos buscando maximizar la métrica Recall pero manteniendo un mínimo valor para la métrica Precision, que recibiremos como parámetro.

In [ ]:
evaluate_models_script_file = 'code/evaluate_models.py'


In [ ]:
%%writefile $evaluate_models_script_file
import argparse
import pickle
import os
import json
import tarfile
import pandas as pd
import numpy as np

from sklearn.metrics import precision_recall_curve

def load_model(file, model_file='model.pkl'):
    if file.endswith('tar.gz'):
        with tarfile.open(file, 'r:gz') as tar:
            for name in tar.getnames():
                if name == model_file:
                    f = tar.extractfile(name)
                    return pickle.load(f)
            return None
    elif file.endswith('pkl'):
        with open(file, 'rb') as f:
            return pickle.load(f)
    else:
        return None

if __name__=='__main__':
    script_name = os.path.basename(__file__)
    print(f'INFO: {script_name}: Iniciando la evaluación de los modelos')
    
    parser = argparse.ArgumentParser()
    parser.add_argument('--algos', type=str, required=True)
    parser.add_argument('--min-precision', type=float, required=True)    
    parser.add_argument('--test-data-file', type=str, required=True)
    parser.add_argument('--test-target-file', type=str, required=True)
    parser.add_argument('--thresholds-file', type=str, required=True)   
    parser.add_argument('--metrics-report-file', type=str, required=True)    
    
    args, _ = parser.parse_known_args()    
    
    print(f'INFO: {script_name}: Parámetros recibidos: {args}')
    
    input_path = '/opt/ml/processing/input'
    output_path = '/opt/ml/processing/output'
    
    # Cargar datasets
    test_target_path = os.path.join(input_path, 'target', args.test_target_file)     
    test_target = pd.read_csv(test_target_path)
    
    test_data_path = os.path.join(input_path, 'data', args.test_data_file)     
    test_data = pd.read_csv(test_data_path)
    
    # Umbrales de decision por algoritmo
    algo_metrics = {'Algorithm':[], 'Threshold':[], 'Precision':[], 'Recall':[]}
    
    metrics_report = {}
    
    algos = args.algos.split(',')
    for algo in algos:
        model_path = os.path.join(input_path, algo, 'model.tar.gz')         

        # Carga modelo en memoria
        print(f'Cargando modelo: {model_path}')
        clf = load_model(model_path)
        
        # Obtiene predicciones con dataset para pruebas
        predictions = clf.predict_proba(test_data)[:, 1]
        
        # Busca umbral de decision
        precision, recall, thresholds = precision_recall_curve(test_target, predictions)
        operating_point_idx = np.argmax(precision>=args.min_precision)
        
        algo_metrics['Threshold'].append(thresholds[operating_point_idx])
        algo_metrics['Precision'].append(precision[operating_point_idx])
        algo_metrics['Recall'].append(recall[operating_point_idx])
        algo_metrics['Algorithm'].append(algo)
        
        metrics_report[algo] = {
            'precision': {'value': precision[operating_point_idx], 'standard_deviation': 'NaN'},
            'recall': {'value': recall[operating_point_idx], 'standard_deviation': 'NaN'}}
         
    
    # Guardar Thresholds    
    metrics = pd.DataFrame(algo_metrics)
    print(f'INFO: {script_name}: Thresholds encontrados')
    print(metrics)
    metrics.to_csv(os.path.join(output_path, args.thresholds_file), index=False)    
    
    # Guardar reporte de metricas para cada modelo
    for algo in metrics_report:
        with open(os.path.join(output_path, f'{algo}_metrics.json'), 'w') as f:
            json.dump({'binary_classification_metrics':metrics_report[algo]},f)        

    
    print(f'INFO: {script_name}: Finalizando la evaluación de los modelos')


Subimos el script a un bucket de Amazon S3 para poder utilizarlo con el Processing Job

In [ ]:
evaluate_models_script_path = sagemaker_utils.upload(evaluate_models_script_file, f's3://{bucket}/{code_prefix}')


Ya que tenemos el script listo en S3, podemos pasar a crear el Processing Job

In [ ]:
evaluation_processor = Processor(
    image_uri=docker_images['Processing']['image_uri'],
    role=sagemaker_role,
    instance_count=1,
    instance_type='ml.m5.large',
    entrypoint=['python3',f'/opt/ml/processing/input/code/{os.path.basename(evaluate_models_script_file)}'],
    volume_size_in_gb=5,
    max_runtime_in_seconds=60*60*2)# dos horas 


ejecutamos el proceso con el método run(), proporcionando los siguientes parámetros:

inputs – rutas de las ubicaciones en Amazon S3 de los archivos de entrada a ser utilizados, en este caso el dataset para pruebas, así como los archivos de predicciones de cada uno de los modelos y el programa Python a ejecutar dentro del contenedor
outputs – ruta para guardar el resultado del proceso, en este caso los umbrales para cada modelo
arguments – parámetros del proceso que son pasados cómo argumentos de la línea de comandos al programa Python que se ejecuta dentro del contenedor

In [ ]:
thresholds_file = 'thresholds.csv'
metrics_report_file = 'metrics_report.json'

eval_parameters = {
    'inputs':[ProcessingInput(
                  input_name='code',
                  source=evaluate_models_script_path,
                  destination='/opt/ml/processing/input/code'),
              ProcessingInput(
                  source=sagemaker_utils.get_processor_output_path(processor, 'test_target'), 
                  destination='/opt/ml/processing/input/target'),
              ProcessingInput(
                  source=sagemaker_utils.get_processor_output_path(processor, 'test_data'), 
                  destination='/opt/ml/processing/input/data'),
              ProcessingInput(
                  source=sagemaker_utils.get_tuner_best_model_artifacts_path(tuners['GradientBoosting']), 
                  destination='/opt/ml/processing/input/GradientBoosting'),
              ProcessingInput(
                  source=sagemaker_utils.get_tuner_best_model_artifacts_path(tuners['RandomForest']),
                  destination='/opt/ml/processing/input/RandomForest'),
              ProcessingInput(
                  source=sagemaker_utils.get_tuner_best_model_artifacts_path(tuners['ExtraTrees']), 
                  destination='/opt/ml/processing/input/ExtraTrees')],
    'outputs':[ProcessingOutput(
                   output_name='eval',
                   source='/opt/ml/processing/output',
                   destination=f's3://{bucket}/{eval_prefix}')],
    'arguments':['--algos', ','.join(estimators.keys()),
                 '--min-precision', '0.85',
                 '--test-data-file', test_data_file,
                 '--test-target-file', test_target_file,
                 '--thresholds-file', thresholds_file,
                 '--metrics-report-file', metrics_report_file]}

evaluation_processor.run(**eval_parameters)


Seleccionar el mejor modelo

In [ ]:
thresholds_path = sagemaker_utils.get_processor_output_path(evaluation_processor,'eval')
metrics = sagemaker_utils.read_csv(f'{thresholds_path}/{thresholds_file}')

max_recall = metrics[metrics['Recall']==metrics['Recall'].max()]
best_model_found = max_recall.loc[max_recall['Precision'].idxmax()]

best_model_found
